# データの特徴

In [ ]:
!pip install optuna optuna-integration

# import lightgbm as lgb
import optuna.integration.lightgbm as lgb
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd

from sklearn import metrics
import matplotlib.pyplot as plt

import optuna

import os
import re

import seaborn as sns
from sklearn.metrics import f1_score, roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay

In [ ]:
file_path = '/Users/shigeyuki-t/Desktop/output.csv'
dataset = pd.read_csv(file_path)
dataset.head(3)

In [ ]:
dataset.info()

In [ ]:
dataset.describe()

# フォルダ・ファイルの作成

In [ ]:
# フォルダ・ファイルの作成
remove_name = ['sample1115@gmail.com','test1124@gmail.com','0gt38r252z23g5v@ezweb.ne.jp',
               'shigeyuki.taira@gmail.com','kitajima.koki.kp8@naist.ac.jp','0gt38r252z23g5v@ezweb.ne.jp',
               'yokota.eiko.yg4@bs.naist.jp','sekiguchi.hiroka.si1@naist.ac.jp','mushaffarasyidridha@gmail.com',
              'naka.taisuke.nt3@bs.naist.jp','valeriemegan10@yahoo.com'] #valeriemegan10@yahoo.comは許可取れてない
dataset = dataset[~dataset['userEmail'].isin(remove_name)]
print(dataset['userEmail'].nunique())
print(dataset['userEmail'].value_counts())

unique_ids = dataset['userEmail'].unique()
save_location = '/Users/shigeyuki-t/Desktop/Experiment_2/analysis/subject/'

for userEmail in unique_ids:
    # フォルダ作成
    folder_name = f'{userEmail}'
    folder_path = os.path.join(save_location, folder_name)
    
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f'created:b {folder_path}')
    else:
        print(f'already exists: {folder_path}')
    # csv作成
    csv_name =  f'{userEmail}' + '.csv'
    csv_path = '/Users/shigeyuki-t/Desktop/Experiment_2/analysis/subject/' + f'{userEmail}' + '/' 
    mail_dataset = dataset[dataset['userEmail'] == f'{userEmail}']
    csv_create_location = os.path.join(csv_path, csv_name)
    mail_dataset.to_csv(csv_create_location, index = True)
    
    if not os.path.exists(csv_create_location):
        os.makedirs(csv_create_location)
        print(f'created:b {csv_create_location}')
    else:
        print(f'already exists: {csv_create_location}')

# データ成形

In [ ]:
from sklearn.preprocessing import StandardScaler

for userEmail in unique_ids:
    user_name = userEmail
    file_path = '/Users/shigeyuki-t/Desktop/Experiment_2/analysis/subject/' + f'{user_name}/' + f'{user_name}' + '.csv'

    if os.path.exists(file_path):
        refined_data = pd.read_csv(file_path)
    else:
        print(f"File not found: {file_path}")
        continue

    refined_data = refined_data.drop(columns=['Unnamed: 0'])

    # UNIXタイムスタンプをdatetime形式に変換・ソート
    refined_data['dateTime'] = pd.to_datetime(refined_data['dateTime'], unit='s')
    refined_data = refined_data.sort_values(by='dateTime')
    refined_data['dateTime_numeric'] = refined_data['dateTime'].astype('int64') // 10**9  # 1/21追加分

    # isCorrectを1/0で分けた後次のtaskの値を参照
    refined_data['isCorrect'] = refined_data['isCorrect'].replace({True: 1, False: 0})
    tasks = sorted(refined_data['task'].unique())  # taskの順番を取得
    for i in range(len(tasks) - 1):
        current_task = tasks[i]
        next_task = tasks[i + 1]
        # 現在のtaskのisCorrectを次のtaskのisCorrectに反映
        refined_data.loc[refined_data['task'] == current_task, 'isCorrect'] = \
        refined_data.loc[refined_data['task'] == next_task, 'isCorrect'].values[0]

    # 指定された ["question"] の ["isCorrect"] を反転
    flip_questions = [49, 84, 111, 136, 150]
    refined_data.loc[refined_data['question'].isin(flip_questions), 'isCorrect'] = \
        refined_data.loc[refined_data['question'].isin(flip_questions), 'isCorrect'].apply(lambda x: 1 - x)

    # 指定された ["question"] の ["isCorrect"] を一律1に変更
    set_to_one_questions = [26, 28, 36, 80]
    refined_data.loc[refined_data['question'].isin(set_to_one_questions), 'isCorrect'] = 1


    min_task = 3
    max_task = refined_data['task'].max()
    refined_data = refined_data[(refined_data['task'] > min_task) & (refined_data['task'] <= max_task)]

    # "pre_correct_rate"列の作成
    def calculate_pre_correct_rate(refined_data):
        # 新しい列を初期化
        refined_data['pre_correct_rate'] = 0.0

        # 全データを task ごとにループ
        tasks = sorted(refined_data['task'].unique())
        for n in tasks:
            # 現在の task の範囲を計算 (n-4~n)
            lower_bound = max(n - 10, min(tasks))  # 下限がデータの最小 task を超えないように調整
            upper_bound = n-1

            # 該当する範囲のデータを抽出
            task_range_data = refined_data[(refined_data['task'] >= lower_bound) & (refined_data['task'] <= upper_bound)]

            # "isCorrect"==1 の task 群の数を計算
            correct_task_count = task_range_data[task_range_data['isCorrect'] == 1]['task'].nunique()

            # 範囲内のタスク数を計算
            total_task_count = len(task_range_data['task'].unique())

            # 正解率を計算 (正解タスク数 / 範囲内のタスク数)
            correct_rate = correct_task_count / total_task_count if total_task_count > 0 else 0

            # 現在の task の行に対して "correct_rate" を設定
            refined_data.loc[refined_data['task'] == n, 'pre_correct_rate'] = correct_rate

        return refined_data

    # correct_rate を計算してデータに追加
    refined_data = calculate_pre_correct_rate(refined_data)

    # "future_correct_rate"列の作成
    def calculate_future_correct_rate(refined_data):
        # 新しい列を初期化
        refined_data['future_correct_rate'] = 0.0

        # 全データを task ごとにループ
        tasks = sorted(refined_data['task'].unique())
        for n in tasks:
            # 現在の task の範囲を計算 (n-4~n)
            lower_bound = max(n, min(tasks))  # 下限がデータの最小 task を超えないように調整
            upper_bound = n+9

            # 該当する範囲のデータを抽出
            task_range_data = refined_data[(refined_data['task'] >= lower_bound) & (refined_data['task'] <= upper_bound)]

            # "isCorrect"==1 の task 群の数を計算
            correct_task_count = task_range_data[task_range_data['isCorrect'] == 1]['task'].nunique()

            # 範囲内のタスク数を計算
            total_task_count = len(task_range_data['task'].unique())

            # 正解率を計算 (正解タスク数 / 範囲内のタスク数)
            correct_rate = correct_task_count / total_task_count if total_task_count > 0 else 0

            # 現在の task の行に対して "correct_rate" を設定
            refined_data.loc[refined_data['task'] == n, 'future_correct_rate'] = correct_rate

        return refined_data

    # correct_rate を計算してデータに追加
    refined_data = calculate_future_correct_rate(refined_data)

    print(f"pre:{refined_data["pre_correct_rate"].head()}")
    print(f"future:{refined_data["future_correct_rate"].head()}")

    # future_correct_rateに基づいてsatisficingを設定
    refined_data["satisficing"] = (refined_data["future_correct_rate"] < 0.4).astype(int)

    # カテゴリ変数をダミー変数に変換
    refined_data = pd.get_dummies(refined_data, columns=['event'])
    required_columns = ['event_response', 'event_scroll', 'event_tapCount']
    for column in required_columns:
        if column not in refined_data.columns:
            refined_data[column] = 0  # 生成されなかったカラムは0で埋める

    # NaNを0で埋める
    refined_data.fillna(0, inplace=True)
    comp_refined_data = refined_data.copy()

    # 数値型列を抽出・# 標準化の適用
    numeric_columns = refined_data.select_dtypes(include=['float64', 'int64']).columns
    numeric_columns = numeric_columns.drop(['task', 'isCorrect','pre_correct_rate','future_correct_rate',"satisficing"])  # 'task'と'isCorrect'列を標準化対象から除外
    scaler = StandardScaler()
    refined_data[numeric_columns] = scaler.fit_transform(refined_data[numeric_columns])

    # 標準化後のデータを保存
    csv_name = 'feature_' + f'{user_name}' + '.csv'
    csv_path = '/Users/shigeyuki-t/Desktop/Experiment_2/analysis/subject/' + f'{user_name}' + '/'
    csv_create_location = os.path.join(csv_path, csv_name)
    if not os.path.exists(csv_path):
        os.makedirs(csv_path)
    refined_data.to_csv(csv_create_location, index=True)

    print(f'File saved: {csv_create_location}')

print(refined_data)

In [ ]:
from scipy.interpolate import make_interp_spline
#適切回答/不良回答の割合を算出
total_bad_count = 0
total_good_count = 0

for userEmail in unique_ids:
    print(userEmail)
    file_path = '/Users/shigeyuki-t/Desktop/Experiment_2/analysis/subject/' + f'{userEmail}' + '/feature_' + f'{userEmail}' + '.csv'

    if os.path.exists(file_path):
        refined_data = pd.read_csv(file_path)
    else:
        print(f"File not found: {file_path}")
        continue

    # 平均正答率
    unique_tasks = refined_data.drop_duplicates(subset="task")
    average_correct_rate = unique_tasks["future_correct_rate"].mean()

    # 平均以下のマーク
    refined_data = refined_data.merge(unique_tasks[['task', 'future_correct_rate']], on='task', suffixes=('', '_unique'))
    refined_data["below_average"] = refined_data["future_correct_rate_unique"] < 0.4

    # # スプライン補間による滑らかな折れ線グラフ
    # x = unique_tasks["task"]
    # y = unique_tasks["pre_correct_rate"]

    # # 補間のための新しいx軸データ生成
    # x_new = np.linspace(x.min(), x.max(), 300)
    # spl = make_interp_spline(x, y, k=3)  # 3次スプライン補間
    # y_smooth = spl(x_new)

    # Plot task-wise correct rate with below-average points highlighted
    plt.figure(figsize=(12, 8))
    sns.lineplot(data=unique_tasks, x="task", y="pre_correct_rate", color="blue", label="Correct Rate")#, marker="o"

    # Highlight below-average points in red
    below_average_data = unique_tasks[unique_tasks["future_correct_rate"] < 0.4]#average_correct_rate
    plt.scatter(below_average_data["task"], below_average_data["pre_correct_rate"], color="red", label="Below Average", zorder=5)

    # Add a horizontal line for the average correct rate
    plt.axhline(y=average_correct_rate, color="green", linestyle="--", label=f"Average Correct Rate ({average_correct_rate:.2f})")

    # Customize plot aesthetics
    plt.title(f"Task-wise Correct Rate with Average Highlight ({userEmail})", fontsize=16)
    plt.xlabel("Task", fontsize=14)
    plt.ylabel("Correct Rate", fontsize=14)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.legend(fontsize=12)
    plt.grid(alpha=0.5)
    plt.tight_layout()
    plt.show()

    # bad_count = 0
    # good_count = 0
    # # 全データを task ごとに処理
    # tasks = refined_data['task'].unique()
    # for task in tasks:
    #     # 現在の task に対応するデータを取得
    #     task_data = refined_data[refined_data['task'] == task]

    #     # isCorrect の 0 (bad) と 1 (good) をそれぞれ判定
    #     if (task_data['isCorrect'] == 0).any():
    #         bad_count += 1  # bad_count を +1
    #     if (task_data['isCorrect'] == 1).any():
    #         good_count += 1  # good_count を +1

    # total_bad_count += bad_count
    # total_good_count += good_count

    # # taskごとの正答率推移を表にする処理
    # def calculate_correct_rate_trend(refined_data):
    #     # taskごとのcorrect_rateの平均を計算
    #     correct_rate_trend = refined_data.groupby('task')['correct_rate'].mean().reset_index()

    #     # カラム名をわかりやすく変更
    #     correct_rate_trend.columns = ['task', 'average_correct_rate']

    #     return correct_rate_trend
    # taskごとの正答率推移を表にする処理（5の倍数のtaskのみ参照）
    # def calculate_correct_rate_trend_for_multiples_of_5(refined_data):
    #     # taskごとのcorrect_rateの平均を計算
    #     correct_rate_trend = refined_data.groupby('task')['correct_rate'].mean().reset_index()

    #     # taskが5の倍数のみフィルタリング
    #     correct_rate_trend = correct_rate_trend[correct_rate_trend['task'] % 10 == 0]

    #     # カラム名をわかりやすく変更
    #     correct_rate_trend.columns = ['task', 'average_correct_rate']

    #     return correct_rate_trend

    # # 正答率の推移を計算
    # correct_rate_trend = calculate_correct_rate_trend_for_multiples_of_5(refined_data)

    # # グラフを描画する処理
    # def plot_correct_rate_trend(correct_rate_trend, user_name):
    #     plt.figure(figsize=(10, 6))
    #     sns.lineplot(data=correct_rate_trend, x='task', y='average_correct_rate', marker='o', color='blue', label='Correct Rate')
    #     plt.title(f'Task-wise Correct Rate Trend ({user_name})', fontsize=16)
    #     plt.xlabel('Task', fontsize=14)
    #     plt.ylabel('Average Correct Rate', fontsize=14)
    #     plt.xticks(fontsize=12)
    #     plt.yticks(fontsize=12)
    #     plt.grid(True, linestyle='--', alpha=0.7)
    #     plt.legend(fontsize=12)
    #     plt.tight_layout()

    #     # グラフを保存
    #     plot_path = '/Users/shigeyuki-t/Desktop/Experiment_2/analysis/subject/' + f'{user_name}/correct_rate_trend.png'
    #     plt.savefig(plot_path)
    #     print(f"Correct rate trend graph saved: {plot_path}")
    #     plt.show()

    # # グラフの描画
    # plot_correct_rate_trend(correct_rate_trend, user_name)


    # 最終的なカウントを出力
    # print(f"Total bad_count: {bad_count}")
    # print(f"Total good_count: {good_count}")

# print(total_bad_count)
# print(total_good_count)

# 特徴量クラスタリング

In [ ]:
# from sklearn.cluster import KMeans
# from sklearn.decomposition import PCA

# # 特徴量によるクラスタリング
user_group_data = []

user_group = [
"amemiya.naoki.ao9@naist.ac.jp",
"kasatani.hikaru.kd6@bs.naist.jp",
"syouta9592@i.softbank.jp",
"nishida.amane.mw6@ms.naist.jp",
"bessho.taisei.bu6@gmail.com",
"maya.sugi0102@gmail.com",
"sannzasannza@gmail.con",
"norilemon811@gmail.com",
"personal@muhammadalqaaf.com",
"niapotter@gmail.com",
"rubaet.ema@gmail.com",
"ito.mana.ik9@ms.naist.jp",
"oyebodeoyewale065@gmail.com", 
"koyama.honami.kg7@naist.ac.jp",
"ilya.maisarah_binti_thariq.in9@naist.ac.jp", 
"reru428@moimoi.re",
"sasaki.rina.su6@g.ext.naist.jp", 
"marina.alki@tum.de",
"yamamoto.yukino.yv8@ms.naist.jp", 
"li.siyuan.lu1@is.naist.jp",
"nakamura.kaito.nj3@is.naist.jp", 
"zjnnyyj@outlook.jp",
"mspamungkas@icloud.com", 
"m.kojima.kn6@naist.ac.jp",
"micnaist@gmail.com",
"nandakumar.ardra.na1@bs.naist.jp",
"tester11_1@gmail.com"
]

for sensor_id in unique_ids:
    file_path = f'/Users/shigeyuki-t/Desktop/Experiment_2/analysis/subject/{sensor_id}/feature_{sensor_id}.csv'
    df = pd.read_csv(file_path, index_col=0)
    # 必要な列のみ抽出
    remove_feature = ['dateTime', 'userEmail', 'screenW', 'screenH']
    df = df.drop(columns=remove_feature, errors='ignore')
    df_mean = df.mean()  # 各被験者の特徴量の平均値を計算
    user_group_data.append(df_mean)

# 特徴量データを1つのデータフレームに統合
feature_df = pd.DataFrame(user_group_data, index= unique_ids)

# print(type(feature_df))
# print(feature_df.isnull().sum())
# print(np.isinf(feature_df).sum().sum())

# 欠損値や無限大を処理
feature_df = feature_df.fillna(0)  # NaNを0で埋める
feature_df = feature_df.replace([np.inf, -np.inf], 0)  # 無限大を0で埋める

# 標準化
scaler = StandardScaler()
scaled_features = scaler.fit_transform(feature_df)

# k-meansクラスタリングでグループ分け（k=3）
n_clusters = 4
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(scaled_features)

# グループ情報を出力
grouped_users = {i: [] for i in range(n_clusters)}
for user, label in zip(user_group, cluster_labels):
    grouped_users[label].append(user)

print("Clustered groups:")
print(grouped_users)

# PCAで2次元に縮約
pca = PCA(n_components=2)
pca_components = pca.fit_transform(scaled_features)

# クラスタリング結果の可視化
plt.figure(figsize=(8, 6))
for i in range(n_clusters):
    plt.scatter(pca_components[cluster_labels == i, 0], pca_components[cluster_labels == i, 1], label=f'Cluster {i}', s=50)

# 軸ラベルとタイトルの設定
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('K-means Clustering of User Features')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import NearestNeighbors


# 標準化
scaler = StandardScaler()
scaled_features = scaler.fit_transform(feature_df)

# k-distanceプロットを作成
neighbors = NearestNeighbors(n_neighbors=5)
neighbors_fit = neighbors.fit(scaled_features)
distances, indices = neighbors_fit.kneighbors(scaled_features)

distances = np.sort(distances[:, 4], axis=0)  # 5番目の最近傍距離を取得
plt.plot(distances)
plt.xlabel("Points sorted by distance")
plt.ylabel("5th Nearest Neighbor Distance")
plt.title("k-distance Plot")
plt.show()

# DBSCANクラスタリング
dbscan = DBSCAN(eps=5.0, min_samples=5)  # epsとmin_samplesはデータに応じて調整
cluster_labels = dbscan.fit_predict(scaled_features)

# クラスタ情報を出力
unique_labels = set(cluster_labels)
grouped_users_dbscan = {label: [] for label in unique_labels}
for user, label in zip(user_group, cluster_labels):
    grouped_users_dbscan[label].append(user)

print("DBSCAN Clusters:")
print(grouped_users_dbscan)

# PCAで2次元に縮約
pca = PCA(n_components=2)
pca_components = pca.fit_transform(scaled_features)

# クラスタリング結果の可視化
plt.figure(figsize=(8, 6))
for label in unique_labels:
    if label == -1:
        # -1はノイズ（クラスタに属さないデータ）
        plt.scatter(pca_components[cluster_labels == label, 0], pca_components[cluster_labels == label, 1],
                    label='Noise', color='gray', s=50)
    else:
        plt.scatter(pca_components[cluster_labels == label, 0], pca_components[cluster_labels == label, 1],
                    label=f'Cluster {label}', s=50)

# 軸ラベルとタイトルの設定
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('DBSCAN Clustering of User Features')
plt.legend()
plt.grid(True)
plt.show()


# 回答品質によって特徴量の分布が変わるか

In [ ]:
# 回答品質の正常時と低下時の特徴量の分布をか確認
from scipy.stats import ttest_ind

group_0 = comp_refined_data[comp_refined_data["satisficing"] == 0]
group_1 = comp_refined_data[comp_refined_data["satisficing"]  == 1]

# 比較する列名のリスト
columns_to_compare = ["question", "rightCount", "leftCount", "task", "pre_correct_rate"]

for column in columns_to_compare:
    # group_1 と group_0 の指定した列のデータを抽出
    data_group_0 = group_0[column]
    data_group_1 = group_1[column]
    
    
    # 箱ひげ図を作成
    fig, ax = plt.subplots(figsize=(10, 6))
    # 中央値を太く
    medianprops = dict(color='red', linewidth=2)
    
    # group_1 と group_0 のデータをまとめて箱ひげ図を作成
    ax.boxplot([data_group_0,data_group_1], patch_artist=False,  # 色なし
               labels=['Group 0','Group 1'],
               medianprops=medianprops)
    
    # タイトルとラベルを設定
    ax.set_title(f'Comparison of "{column}" between Group 0 and Group 01')
    ax.set_ylabel(column)
    ax.grid(False)
    
    # レイアウトを整えて表示
    plt.tight_layout()
    plt.show()

# モデル設計

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import TimeSeriesSplit

import lightgbm as lgb

optuna.logging.set_verbosity(optuna.logging.WARNING)

def objective(trial, train_dataset, test_dataset):
    params = {
        'objective': 'binary',
        'metric': 'binary_error',
        'num_leaves': trial.suggest_int('num_leaves', 8, 32),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 50, 150),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 5, 15),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 5, 15),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        # 'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 0.1),
        'verbosity': -1,
        'early_stopping_rounds':10,
        'verbose_eval':False,
        "feature_pre_filter": False,
        "is_unbalance": True
    }
    # モデルの作成と訓練
    model = lgb.train(params, train_dataset, num_boost_round=1000, valid_sets=test_dataset)

    y_pred = model.predict(X_test, num_iteration=model.best_iteration)
    y_pred = [1 if pred > 0.5 else 0 for pred in y_pred]
    return accuracy_score(y_test, y_pred)

user_group1 = [
"amemiya.naoki.ao9@naist.ac.jp",
"kasatani.hikaru.kd6@bs.naist.jp",
"syouta9592@i.softbank.jp",

"nishida.amane.mw6@ms.naist.jp",
"bessho.taisei.bu6@gmail.com",
"maya.sugi0102@gmail.com",
"sannzasannza@gmail.con",
"norilemon811@gmail.com",
"personal@muhammadalqaaf.com",
"niapotter@gmail.com",
"rubaet.ema@gmail.com",
"ito.mana.ik9@ms.naist.jp",
"oyebodeoyewale065@gmail.com",
"koyama.honami.kg7@naist.ac.jp",
"ilya.maisarah_binti_thariq.in9@naist.ac.jp",
"reru428@moimoi.re",

"sasaki.rina.su6@g.ext.naist.jp",

"marina.alki@tum.de",
"yamamoto.yukino.yv8@ms.naist.jp",

"li.siyuan.lu1@is.naist.jp",
"nakamura.kaito.nj3@is.naist.jp",

"zjnnyyj@outlook.jp",
"mspamungkas@icloud.com",
"m.kojima.kn6@naist.ac.jp",
"micnaist@gmail.com",
"nandakumar.ardra.na1@bs.naist.jp",
"tester11_1@gmail.com"
]

cluster_user =[
    # 'nishida.amane.mw6@ms.naist.jp', 
    # 'norilemon811@gmail.com', 
    # 'personal@muhammadalqaaf.com', 
    # 'niapotter@gmail.com', 
    # 'rubaet.ema@gmail.com', 
    # 'oyebodeoyewale065@gmail.com', 
    # 'koyama.honami.kg7@naist.ac.jp', 
    # 'sasaki.rina.su6@g.ext.naist.jp', 
    # 'marina.alki@tum.de', 
    # 'nandakumar.ardra.na1@bs.naist.jp'
              ]

all_user_test = []
all_user_pred = []
all_user_feature_importances = []

for sensor_id in user_group1 :
    user_name = sensor_id
    file_path = f'/Users/shigeyuki-t/Desktop/Experiment_2/analysis/subject/{user_name}/feature_{user_name}.csv'
    df = pd.read_csv(file_path, index_col=0)

#     print(df.columns)

    remove_feature = ['dateTime','userEmail',]# 'question','leftCount','rightCount','screenW', 'screenH'
    df = df.drop(columns=remove_feature)

    # 欠損値や無限大を処理
    df = df.fillna(0)
    df = df.replace([np.inf, -np.inf], 0)

    features = set(df.columns) - {"satisficing",'future_correct_rate'}
    features = list(features)
    target = "satisficing"

    predictions = []
    train_accuracies = []
    test_accuracies = []

    # 全体の予測結果を保存するリスト
    all_y_test = []  # 実際のラベル
    all_y_pred = []  # 予測結果
    all_feature_importances = []

    for n in range(5, 150):
        # 'task'列を基準にデータを分割
        # train_data = df[df['task'].isin(range(n-10, n+1))]
        # if train_data.empty:
        #     print(f"訓練データが空のため、範囲を調整（n={n}）")
        #     train_data = df[df['task'].isin(range(n-(n-1), n+1))]
        # test_data = df[df['task'].isin(range(n+1, n+6))]
        train_data = df[df['task'].isin(range(n+1))]
        test_data = df[df['task']==n+1]

        X_train = train_data[features]
        y_train = train_data[target]

        X_test = test_data[features]
        y_test = test_data[target]

        # 特徴量やラベルが空の場合はスキップ
        if X_train.empty or X_test.empty or y_train.empty or y_test.empty:
            continue

        # クラスが２つ以上あるか
        if len(y_train.unique()) > 1:
            # LightGBM用データセットの作成
            train_dataset = lgb.Dataset(X_train, label=y_train)
            test_dataset = lgb.Dataset(X_test, label=y_test, reference=train_dataset)

            study = optuna.create_study(direction='maximize')
            study.optimize(lambda trial: objective(trial, train_dataset, test_dataset), n_trials=50)

            best_params = study.best_params
            best_params.update({'objective': 'binary', 'metric': 'binary_error', 'verbosity': -1})


            params = {
                'objective': 'binary',
                'metric': 'binary_error',
                'num_leaves': 16,
                'min_data_in_leaf': 100,
                'lambda_l1': 10.0,
                'lambda_l2': 10.0,
                'feature_fraction': 0.8,
                'bagging_fraction': 0.8,
                'bagging_freq': 5,
                'early_stopping_rounds': 10,
                'verbosity': -1,
                'verbose_eval':False
            }

            # モデルの作成と訓練
            model = lgb.train(best_params, train_dataset, num_boost_round=1000, valid_sets=test_dataset)
            #model = lgb.train(params, train_dataset, num_boost_round=1000, valid_sets=test_dataset)

            #特徴量重要度を取得
            importance = model.feature_importance()
            importance_df = pd.DataFrame({'Feature': features, 'Importance': importance})
            all_feature_importances.append(importance_df)
            all_user_feature_importances.append(importance_df)

            # 保存
            model_path = '/Users/shigeyuki-t/Desktop/Experiment_2/analysis/subject/' + f'{user_name}/'
            if not os.path.exists(model_path):
                    os.makedirs(model_path)
            model.save_model(os.path.join(model_path, f'model_{n}.txt'))

            # 訓練データでの予測（過学習チェック）
            y_train_pred = model.predict(X_train, num_iteration=model.best_iteration)
            y_train_pred_binary = [1 if pred > 0.5 else 0 for pred in y_train_pred]

            # テストデータでの予測
            y_pred = model.predict(X_test, num_iteration=model.best_iteration)
            y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]

            # 1タスクごとの予測値を追加
            all_y_test.append(y_test.tolist()[0])
            all_y_pred.append(y_pred_binary[0])
            all_user_test.append(y_test.tolist()[0])
            all_user_pred.append(y_pred_binary[0])

            # 訓練データとテストデータの精度を計算（過学習チェック）
            train_accuracy = accuracy_score(y_train, y_train_pred_binary)
            test_accuracy = accuracy_score(y_test, y_pred_binary)
            train_accuracies.append(train_accuracy)
            test_accuracies.append(test_accuracy)

        else:
            # 最頻値を使う場合
            if not y_train.empty and not y_train.mode().empty:
                y_pred = [y_train.mode()[0]]
            else:
                y_pred = [0]
                # 予測結果を追加
                all_y_test.append(y_test.tolist()[0])
                all_y_pred.append(y_pred[0])
                all_user_test.append(y_test.tolist()[0])
                all_user_pred.append(y_pred[0])

        # 予測結果を保存
#         predictions.extend(y_pred if len(y_pred) > 0 else [1])
        predictions.extend(list(y_pred) if len(y_pred) > 0 else [1])

    # 訓練データとテストデータの精度を表示
    # print(f"train_accuracies:{train_accuracies}")
    # ones_count = train_accuracies.count(1.0)
    # print(f"訓練データの1.0 の数: {ones_count}")
    # print(f"test_accuracies: {test_accuracies}")
    # t_ones_count = test_accuracies.count(1.0)
    # print(f"テストデータの1.0 の数: {t_ones_count}")
    # print(f"test_accuracies: {sum(test_accuracies)}/{len(test_accuracies)}")

    # print(f"推定値: {all_y_pred}")

    # plt.figure(figsize=(8, 6))
    # sns.histplot(all_y_pred, kde=True, bins=20, color='blue')
    # plt.xlabel("Predicted Probability")
    # plt.ylabel("Frequency")
    # plt.title("Distribution of Predicted Probabilities")
    # plt.grid()
    # plt.show()
    # 混同行列の描画を変更
    cm = confusion_matrix(all_y_test, all_y_pred)
    labels = [0,1]
    plt.figure(figsize=(6, 6))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title(f'Confusion Matrix')
    plt.colorbar()
    tick_marks = range(len(labels))
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)
    # 各セルに値を表示
    thresh = cm.max() / 2
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'),
                    horizontalalignment="center",
                    color="white" if cm[i, j] > thresh else "black")
    plt.xlabel('Predict')
    plt.ylabel('Actual')
    plt.tight_layout()
    plt.show()
    # ROC曲線の描画を変更
    # fpr, tpr, _ = roc_curve(all_y_test, all_y_pred)
    # roc_auc = auc(fpr, tpr)
    # plt.figure()
    # plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
    # plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    # plt.xlabel('False Positive Rate')
    # plt.ylabel('True Positive Rate')
    # plt.title(f'ROC Curve for Task {n}')
    # plt.legend(loc="lower right")
    # plt.show()
    # F1スコアの計算を追加
    # train_f1 = f1_score(y_train, y_train_pred_binary)
    # True Positives, True Negatives, False Positives, False Negativesを取り出す
    TP = cm[0, 0]
    TN = cm[1, 1]
    FP = cm[0, 1]
    FN = cm[1, 0]
    # 精度の計算
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    print(f"Accuracy: {accuracy}")
    test_f1 = f1_score(all_y_test, all_y_pred)
    print(f"Test F1: {test_f1:.2f}")
    # 特徴量重要度の可視化
    combined_importance = pd.concat(all_feature_importances)
    aggregated_importance = combined_importance.groupby('Feature').mean().reset_index()
    aggregated_importance = aggregated_importance.sort_values(by='Importance', ascending=False)
    plt.figure(figsize=(10, 6))
    plt.barh(aggregated_importance['Feature'], aggregated_importance['Importance'], color='skyblue')
    plt.xlabel('Average Importance')
    plt.ylabel('Feature')
    plt.title('Average Feature Importance Across All Models')
    plt.gca().invert_yaxis()  # 特徴量を上から順に表示
    plt.tight_layout()
    plt.show()
    print("Top 10 Features by Average Importance:")
    print(aggregated_importance.head(10))
    
    
    
#全員分の統合
# 混同行列の描画
print("総合評価")
all_cm = confusion_matrix(all_user_test, all_user_pred)
labels = [0,1]
plt.figure(figsize=(6, 6))
plt.imshow(all_cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title(f'Confusion Matrix')
plt.colorbar()
tick_marks = range(len(labels))
plt.xticks(tick_marks, labels, rotation=45)
plt.yticks(tick_marks, labels)
# 各セルに値を表示
thresh = all_cm.max() / 2
for i in range(all_cm.shape[0]):
    for j in range(all_cm.shape[1]):
        plt.text(j, i, format(all_cm[i, j], 'd'),
                horizontalalignment="center",
                color="white" if all_cm[i, j] > thresh else "black")
plt.xlabel('Predict')
plt.ylabel('Actual')
plt.tight_layout()
plt.show()

TP = all_cm[0, 0]
TN = all_cm[1, 1]
FP = all_cm[0, 1]
FN = all_cm[1, 0]
# 精度の計算
all_accuracy = (TP + TN) / (TP + TN + FP + FN)
print(f"Accuracy: {all_accuracy}")
all_f1 = f1_score(all_user_test, all_user_pred)
print(f"F1: {all_f1:.2f}")

# 特徴量重要度の可視化
combined_all_importance = pd.concat(all_user_feature_importances)
aggregated_all_importance = combined_all_importance.groupby('Feature').mean().reset_index()
aggregated_all_importance = aggregated_all_importance.sort_values(by='Importance', ascending=False)
plt.figure(figsize=(10, 6))
plt.barh(aggregated_all_importance['Feature'], aggregated_all_importance['Importance'], color='skyblue')
plt.xlabel('Average Importance')
plt.ylabel('Feature')
plt.title('Average Feature Importance Across All Models')
plt.gca().invert_yaxis()  # 特徴量を上から順に表示
plt.tight_layout()
plt.show()
print("Top 10 Features by Average Importance:")
print(aggregated_all_importance.head(10))

In [ ]:
accuracy = [
    0.879432624113, 0.909090909091, 0.885416666667, 0.921568627451, 0.944055944056,
    0.844444444444, 0.885416666667, 0.911111111111, 0.956834532374, 0.950704225352,
    0.913793103448, 0.842519685039, 0.929203539823, 0.816666666667, 0.944444444444,
    0.867647058824, 0.93023255814, 0.941605839416, 0.918367346939, 0.969465648855,
    0.935483870968, 0.931034482759, 0.888888888889, 0.896551724138, 0.915254237288,
    0.918032786885, 0.85
]
# F1_score = [
#     0.925110132159, 0.948979591837, 0.928104575163, 0.956989247312, 0.966666666667,
#     0.893401015228, 0.917293233083, 0.953125, 0.97619047619, 0.97358490566,
#     0.95, 0.913043478261, 0.961165048544, 0.855263157895, 0.968325791855,
#     0.888888888889, 0.959459459459, 0.967479674797, 0.951219512195, 0.983333333333,
#     0.964912280702, 0.95867768595, 0.925170068027, 0.933333333333, 0.952380952381,
#     0.955357142857, 0.912621359223
# ]
precision = [
    0.913043478261, 0.939393939394, 0.934210526316, 0.967391304348, 0.966666666667,
    0.907216494845, 0.910447761194, 0.945736434109, 0.97619047619, 0.977272727273,
    0.95, 0.905172413793, 0.961165048544, 0.844155844156, 0.963963963964,
    0.923076923077, 0.959459459459, 0.959677419355, 0.951219512195, 0.983333333333,
    0.964912280702, 0.95867768595, 0.931506849315, 0.945945945946, 0.943396226415,
    0.946902654867, 0.959183673469
]
recall = [
    0.9375, 0.958762886598, 0.922077922078, 0.946808510638, 0.966666666667,
    0.88, 0.924242424242, 0.96062992126, 0.97619047619, 0.96992481203,
    0.95, 0.921052631579, 0.961165048544, 0.866666666667, 0.972727272727,
    0.857142857143, 0.959459459459, 0.975409836066, 0.951219512195, 0.983333333333,
    0.964912280702, 0.95867768595, 0.918918918919, 0.921052631579, 0.961538461538,
    0.963963963964, 0.87037037037
]

# Accuracy
plt.figure(figsize=(6, 4))
plt.hist(accuracy, bins=10, color='skyblue', edgecolor='black')
plt.title('Accuracy Distribution')
plt.xlabel('Accuracy')
plt.ylabel('Frequency')
plt.show()

# # F1-Score
# plt.figure(figsize=(6, 4))
# plt.hist(f1_score, bins=10, color='orange', edgecolor='black')
# plt.title('F1-Score Distribution')
# plt.xlabel('F1-Score')
# plt.ylabel('Frequency')
# plt.show()

# Precision
plt.figure(figsize=(6, 4))
plt.hist(precision, bins=10, color='green', edgecolor='black')
plt.title('Precision Distribution')
plt.xlabel('Precision')
plt.ylabel('Frequency')
plt.show()

# Recall
plt.figure(figsize=(6, 4))
plt.hist(recall, bins=10, color='red', edgecolor='black')
plt.title('Recall Distribution')
plt.xlabel('Recall')
plt.ylabel('Frequency')
plt.show()

# モデル設計2

In [ ]:
# # 回答品質の正常時と低下時の特徴量の分布をか確認
# group_1 = refined_data[refined_data[”satisficing”] == 1]
# group_0 = refined_data[refined_data[”satisficing”] == 0]

# all_group_data = []

# user_group = [
# "amemiya.naoki.ao9@naist.ac.jp",
# "kasatani.hikaru.kd6@bs.naist.jp",
# "syouta9592@i.softbank.jp",
# "nishida.amane.mw6@ms.naist.jp",
# "bessho.taisei.bu6@gmail.com",
# "maya.sugi0102@gmail.com",
# "sannzasannza@gmail.con",
# "norilemon811@gmail.com",
# "personal@muhammadalqaaf.com",
# "niapotter@gmail.com",
# "rubaet.ema@gmail.com",
# "ito.mana.ik9@ms.naist.jp",
# "oyebodeoyewale065@gmail.com", 
# "koyama.honami.kg7@naist.ac.jp",
# "ilya.maisarah_binti_thariq.in9@naist.ac.jp", 
# "reru428@moimoi.re",
# "sasaki.rina.su6@g.ext.naist.jp", 
# "marina.alki@tum.de",
# "yamamoto.yukino.yv8@ms.naist.jp", 
# "li.siyuan.lu1@is.naist.jp",
# "nakamura.kaito.nj3@is.naist.jp", 
# "zjnnyyj@outlook.jp",
# "mspamungkas@icloud.com", 
# "m.kojima.kn6@naist.ac.jp",
# "micnaist@gmail.com",
# "nandakumar.ardra.na1@bs.naist.jp",
# "tester11_1@gmail.com"
# ]

# for sensor_id in unique_ids:
#     file_path = f'/Users/shigeyuki-t/Desktop/Experiment_2/analysis/subject/{sensor_id}/feature_{sensor_id}.csv'
#     df = pd.read_csv(file_path, index_col=0)
#     # 必要な列のみ抽出
#     remove_feature = ['dateTime', 'userEmail', 'screenW', 'screenH']
#     df = df.drop(columns=remove_feature, errors='ignore')
#     # df_mean = df.mean()  # 各被験者の特徴量の平均値を計算
#     all_group_data.append(df)

# # 特徴量データを1つのデータフレームに統合
# feature_comparison_df = pd.DataFrame(all_group_data, index= unique_ids)
# print(feature_comparison_df)
# # good_user_group = (refined_data[]

# Leave One Out

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
#最初にkmeanで作ったクラスタ
cluster1 = [
    'nishida.amane.mw6@ms.naist.jp',
    'norilemon811@gmail.com',
    'personal@muhammadalqaaf.com',
    'niapotter@gmail.com',
    'rubaet.ema@gmail.com',
    'oyebodeoyewale065@gmail.com',
    'koyama.honami.kg7@naist.ac.jp',
    'sasaki.rina.su6@g.ext.naist.jp',
    'marina.alki@tum.de',
    'nandakumar.ardra.na1@bs.naist.jp'
]

cluster2 = [
    'amemiya.naoki.ao9@naist.ac.jp',
    'kasatani.hikaru.kd6@bs.naist.jp',
    'syouta9592@i.softbank.jp',
    'bessho.taisei.bu6@gmail.com',
    'maya.sugi0102@gmail.com',
    'norilemon811@gmail.com',
    'personal@muhammadalqaaf.com',
    'niapotter@gmail.com',
    'koyama.honami.kg7@naist.ac.jp',
    'ilya.maisarah_binti_thariq.in9@naist.ac.jp',
    'reru428@moimoi.re',
    'sasaki.rina.su6@g.ext.naist.jp',
    'marina.alki@tum.de',
    'yamamoto.yukino.yv8@ms.naist.jp',
    'li.siyuan.lu1@is.naist.jp',
    'zjnnyyj@outlook.jp',
    'tester11_1@gmail.com'
    ]

cluster3 = [
"amemiya.naoki.ao9@naist.ac.jp",
"kasatani.hikaru.kd6@bs.naist.jp",
"syouta9592@i.softbank.jp",
"nishida.amane.mw6@ms.naist.jp",
"bessho.taisei.bu6@gmail.com",
"maya.sugi0102@gmail.com",
"sannzasannza@gmail.con",
"norilemon811@gmail.com",
"personal@muhammadalqaaf.com",
"niapotter@gmail.com",
"rubaet.ema@gmail.com",
"ito.mana.ik9@ms.naist.jp",
"oyebodeoyewale065@gmail.com",
"koyama.honami.kg7@naist.ac.jp",
"ilya.maisarah_binti_thariq.in9@naist.ac.jp",
"reru428@moimoi.re",
"sasaki.rina.su6@g.ext.naist.jp",
"marina.alki@tum.de",
"yamamoto.yukino.yv8@ms.naist.jp",
"li.siyuan.lu1@is.naist.jp",
"nakamura.kaito.nj3@is.naist.jp",
"zjnnyyj@outlook.jp",
"mspamungkas@icloud.com",
"m.kojima.kn6@naist.ac.jp",
"micnaist@gmail.com",
"nandakumar.ardra.na1@bs.naist.jp",
"tester11_1@gmail.com"
]

#DBSCANで作ったクラスタ
cluster0 = [
    'amemiya.naoki.ao9@naist.ac.jp', 'kasatani.hikaru.kd6@bs.naist.jp'#, 'bessho.taisei.bu6@gmail.com', 'sannzasannza@gmail.con', 'norilemon811@gmail.com', 'personal@muhammadalqaaf.com', 'niapotter@gmail.com', 'oyebodeoyewale065@gmail.com', 'koyama.honami.kg7@naist.ac.jp', 'ilya.maisarah_binti_thariq.in9@naist.ac.jp', 'reru428@moimoi.re', 'sasaki.rina.su6@g.ext.naist.jp', 'marina.alki@tum.de', 'yamamoto.yukino.yv8@ms.naist.jp', 'nakamura.kaito.nj3@is.naist.jp', 'nandakumar.ardra.na1@bs.naist.jp'
]
final_predictions = {}

all_lopo_user_test = []
all_lopo_user_pred = []

for test_id in cluster3:
    cross_predictions = []
    testfile_path =  f'/Users/shigeyuki-t/Desktop/Experiment_2/analysis/subject/{test_id}/feature_{test_id}.csv'
    test_df = pd.read_csv(testfile_path)
    test_df = test_df.drop(columns = ['userEmail', 'dateTime', 'Unnamed: 0'])
    # print(test_df.columns)

    roc_df = test_df.copy()

    lopo_user_test = []
    lopo_user_pred = []

    # 各modelを使って予測し、isCorrectを推定
    for sensor_id in [user for user in cluster3 if user != test_id]:
    # for sensor_id in cluster1:
        # print(test_id,sensor_id)
        model_path = '/Users/shigeyuki-t/Desktop/Experiment_2/analysis/subject/' + f'{sensor_id}'
        # print(model_path)
        for n in range(1,151):
            try:
                model_file = os.path.join(model_path, f'model_{n}.txt')
                if not os.path.exists(model_file):
                    continue

                load_model = lgb.Booster(model_file=model_file)

                # モデルが期待する特徴量の確認
                expected_columns = load_model.feature_name()
                # print(f"Expected columns for model {n}: {expected_columns}")
                # print(f"Test data columns: {test_df.columns}")
                # モデルに必要な特徴量がテストデータに含まれているか確認
                missing_columns = list(set(expected_columns) - set(test_df.columns))
                extra_columns = list(set(test_df.columns) - set(expected_columns))
                # print(f"Missing columns in test data: {missing_columns}")
                # print(f"Extra columns in test data: {extra_columns}")
                # 足りないカラムを追加する
                for column in missing_columns:
                    test_df[column] = 0
                # 余分なカラムを削除する
                test_df = test_df.drop(columns=extra_columns)
                # 特徴量の順番をモデルに合わせて並べ替える
                test_df = test_df[expected_columns]
                zikkenn = test_df[expected_columns]

                pred = load_model.predict(zikkenn, num_iteration=load_model.best_iteration)
                # print(pred)
                cross_predictions.append(pred)

            except Exception as e:
                print(f"Failed to load model {n}: {e}")
                continue

    if cross_predictions:
        # cross_predictionsをnumpy配列に変換
        predictions_array = np.array(cross_predictions)  # shape: (モデル数, データ数)

        # 各データポイント（列）の平均を計算
        mean_pred = np.mean(predictions_array, axis=0)
        # print(test_df.columns)  # テストデータのカラム名を表示

        # ROC曲線を使用して最適な閾値を計算
        if 'satisficing' in roc_df.columns:
            actual = roc_df['satisficing'].values  # 実際のターゲット値

            # # Precision-Recall 曲線の計算
            # precision, recall, thresholds = precision_recall_curve(actual, mean_pred)
            # # TP を最大化しつつ、FN を最小化するため、高い Recall を確保
            # desired_recall = 0.85  # Recall が 85%以上になるしきい値を選ぶ
            # valid_idx = np.where(recall[:-1] >= desired_recall)[0]  # thresholds に対応するインデックスを取得
            # if len(valid_idx) > 0:
            #     optimal_idx = valid_idx[-1]  # Recall が 85%以上を満たす中で最も高い閾値を選ぶ
            #     optimal_threshold = thresholds[optimal_idx]
            # else:
            #     optimal_threshold = 0.5  # 適切な閾値がない場合はデフォルト
            # print(f"Optimal threshold from Precision-Recall curve: {optimal_threshold:.4f}")

            fpr, tpr, thresholds = roc_curve(actual, mean_pred)
            optimal_idx = np.argmax(tpr - fpr)  # TPR - FPR が最大となる閾値
            optimal_threshold = thresholds[optimal_idx]
            print(f"Optimal threshold from ROC curve: {optimal_threshold:.4f}")

            # 動的閾値を使用した予測結果
            # re_mean_pred = [1 if p > optimal_threshold else 0 for p in mean_pred]
            re_mean_pred = [1 if p > 0.17 else 0 for p in mean_pred]
            # predict_isCorrect カラムを追加
            roc_df["predict_satisficing"] = re_mean_pred

            # 精度を計算
            # correct_predictions = (roc_df["predict_satisficing"] == roc_df["satisficing"]).sum()
            # accuracy = correct_predictions / len(roc_df)
            # print(f"Accuracy: {accuracy:.4f}")

            # # AUCの計算
            # roc_auc = auc(fpr, tpr)
            # print(f"AUC: {roc_auc:.4f}")

            # F1スコアの計算
            f1 = f1_score(roc_df["satisficing"], roc_df["predict_satisficing"])
            print(f"F1 Score: {f1:.4f}")
            # print(lopo_user_test)
            # print(lopo_user_pred)

            # "task"ごとのユニークなリストを取得
            unique_tasks = roc_df["task"].unique()
            # print("unique_tasks:", unique_tasks)
            # print("zikkenn のカラム一覧:", zikkenn.columns)
            # タスクごとの代表値を取得
            for task in unique_tasks:
                # 該当タスクのデータを抽出
                task_data = roc_df[roc_df["task"] == task]
                # actual（isCorrect）の代表値 → 平均または最頻値
                actual_value = task_data["satisficing"].mean()  # 平均値を使用
                # actual_value = task_data["isCorrect"].mode()[0]  # 最頻値を使う場合
                # re_mean_pred（予測値）の代表値 → 平均値を使用
                pred_value = task_data["predict_satisficing"].mean()
                # リストに追加
                # print(f"追加前の lopo_user_test: {lopo_user_test}")
                lopo_user_test.append(actual_value)
                all_lopo_user_test.append(actual_value)
                # print(f"追加後の lopo_user_test: {lopo_user_test}")
                lopo_user_pred.append(pred_value)
                all_lopo_user_pred.append(pred_value)
                # print("実行されてる")

            # 混同行列の計算
            lopo_user_pred = np.array(lopo_user_pred , dtype=int)
            lopo_user_test = np.array(lopo_user_test, dtype=int)
            cm = confusion_matrix(lopo_user_test, lopo_user_pred)
            disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['0', '1'])
            # print(f"Confusion Matrix:\n{cm}")
            # 最終結果を保存
            final_predictions[test_id] = {
                "accuracy": accuracy,
                "optimal_threshold": optimal_threshold,
                "confusion_matrix": cm,
            }

            # 混同行列の可視化
            disp.plot(cmap='Blues', values_format='d')
            plt.title('Confusion Matrix')
            plt.show()

            # # ROC曲線の描画
            # plt.figure(figsize=(8, 6))
            # plt.plot(fpr, tpr, label=f"ROC curve (AUC = {roc_auc:.2f})")
            # plt.scatter(fpr[optimal_idx], tpr[optimal_idx], color='red', label=f"Optimal Threshold = {optimal_threshold:.2f}")
            # plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
            # plt.xlabel('False Positive Rate')
            # plt.ylabel('True Positive Rate')
            # plt.title('ROC Curve')
            # plt.legend()
            # plt.grid()
            # plt.show()

            # 全体の予測分布を可視化
            plt.figure(figsize=(8, 6))
            plt.hist(mean_pred, bins=30, alpha=0.7, label='Predictions')
            plt.axvline(optimal_threshold, color='red', linestyle='--', label='Optimal Threshold')
            plt.xlabel('Prediction Probability')
            plt.ylabel('Frequency')
            plt.title('Prediction Probability Distribution')
            plt.legend()
            plt.grid()
            plt.show()

            print("Dynamic threshold predictions:", re_mean_pred)
        else:
            print("No 'isCorrect' column found in test data; skipping ROC threshold calculation.")
    else:
        print("No predictions available.")

    if "satisficing" in roc_df.columns and "predict_satisficing" in roc_df.columns:
        # isCorrect の値ごとのカウント
        is_correct_counts = roc_df["satisficing"].value_counts()
        print("Counts in isCorrect column:")
        print(is_correct_counts)

        # predict_isCorrect の値ごとのカウント
        predict_is_correct_counts = roc_df["predict_satisficing"].value_counts()
        print("\nCounts in predict_isCorrect column:")
        print(predict_is_correct_counts)

        # 比較を分かりやすく出力
        print("\nComparison of counts:")
        comparison = pd.DataFrame({
            "satisficing": is_correct_counts,
            "predict_satisficing": predict_is_correct_counts
        }).fillna(0)  # 欠損値を 0 で補完
        print(comparison)
    else:
        print("Either isCorrect or predict_isCorrect column is missing.")

for test_id, result in final_predictions.items():
    print(f"Results for {test_id}:")
    print(f"Accuracy: {result['accuracy']:.4f}")
    print(f"Optimal Threshold: {result['optimal_threshold']:.4f}")
    print("Confusion Matrix:")
    print(result["confusion_matrix"])
    print("-" * 50)


if all_lopo_user_test and all_lopo_user_pred:
    print("総合評価")
    # # 予測値（lopo_user_pred）を0/1のラベルに変換
    all_lopo_user_pred_binary = [1 if p > 0.5 else 0 for p in all_lopo_user_pred]

    # # データ型を整数に変換（明示的にintに統一）
    all_lopo_user_test = np.nan_to_num(all_lopo_user_test, nan=0)
    all_lopo_user_test = np.array(all_lopo_user_test, dtype=int)
    all_lopo_user_pred_binary = np.array(all_lopo_user_pred_binary, dtype=int)

    # 予測と実際のラベルが0と1のみに含まれているか確認
    print(set(all_lopo_user_test))
    print(set(all_lopo_user_pred_binary))

    # 不要なラベルをフィルタリング
    all_lopo_user_test = [x for x in all_lopo_user_test if x in [0, 1]]
    all_lopo_user_pred_binary = [x for x in all_lopo_user_pred_binary if x in [0, 1]]

    # 混同行列を計算
    cm_total = confusion_matrix(all_lopo_user_test, all_lopo_user_pred_binary)
    disp_total = ConfusionMatrixDisplay(confusion_matrix=cm_total, display_labels=['0', '1'])
    disp_total.plot(cmap='Blues', values_format='d')
    plt.xlabel('Predict')
    plt.ylabel('Actual')
    plt.title("Confusion Matrix")
    plt.show()



In [ ]:
accuracy = [
0.689189189189,
0.736486486486,
0.679611650485,
0.837837837838,
0.641891891892,
0.550724637681,
0.602739726027,
0.810810810811,
0.763513513514,
0.790540540541,
0.655405405405,
0.797297297297,
0.858108108108,
0.540540540541,
0.777027027027,
0.630136986301,
0.783333333333,
0.871621621622,
0.655172413793,
0.695945945946,
0.878787878788,
0.918918918919,
0.641891891892,
0.716216216216,
0.682432432432,
0.462585034014,
0.837837837838,
]
F1_score = [
0.801724137931,
0.843373493976,
0.8,
0.911764705882,
0.75117370892,
0.686868686869,
0.743362831858,
0.893129770992,
0.853556485356,
0.880308880309,
0.786610878661,
0.878048780488,
0.923636363636,
0.701754385965,
0.859574468085,
0.773109243697,
0.878504672897,
0.923694779116,
0.782608695652,
0.819277108434,
0.935483870968,
0.952755905512,
0.781893004115,
0.829268292683,
0.801687763713,
0.632558139535,
0.911764705882,
]
precision = [
0.781512605042,
0.905172413793,
0.80487804878,
0.932330827068,
0.879120879121,
0.708333333333,
0.75,
0.951219512195,
0.918918918919,
0.934426229508,
0.87037037037,
0.830769230769,
0.920289855072,
0.634920634921,
0.961904761905,
0.630136986301,
0.878504672897,
0.912698412698,
0.837209302326,
0.902654867257,
0.935483870968,
0.916666666667,
0.77868852459,
0.784615384615,
0.904761904762,
0.85,
0.946564885496,
]
recall = [
0.823008849558,
0.789473684211,
0.795180722892,
0.892086330935,
0.655737704918,
0.666666666667,
0.736842105263,
0.841726618705,
0.796875,
0.832116788321,
0.717557251908,
0.931034482759,
0.92700729927,
0.78431372549,
0.776923076923,
1,
0.878504672897,
0.934959349593,
0.734693877551,
0.75,
0.935483870968,
0.991803278689,
0.785123966942,
0.879310344828,
0.719696969697,
0.503703703704,
0.879432624113,
]

# Accuracy
plt.figure(figsize=(6, 4))
plt.hist(accuracy, bins=10, color='skyblue', edgecolor='black')
plt.title('Accuracy Distribution')
plt.xlabel('Accuracy')
plt.ylabel('Frequency')
plt.show()

# F1-Score
plt.figure(figsize=(6, 4))
plt.hist(F1_score, bins=10, color='orange', edgecolor='black')
plt.title('F1-Score Distribution')
plt.xlabel('F1-Score')
plt.ylabel('Frequency')
plt.show()

# Precision
plt.figure(figsize=(6, 4))
plt.hist(precision, bins=10, color='green', edgecolor='black')
plt.title('Precision Distribution')
plt.xlabel('Precision')
plt.ylabel('Frequency')
plt.show()

# Recall
plt.figure(figsize=(6, 4))
plt.hist(recall, bins=10, color='red', edgecolor='black')
plt.title('Recall Distribution')
plt.xlabel('Recall')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# from sklearn.metrics import f1_score, roc_curve, auc

# cross_predictions = []
# clustered_groups = [
#     'nishida.amane.mw6@ms.naist.jp', 
#     'norilemon811@gmail.com', 
#     'personal@muhammadalqaaf.com', 
#     'niapotter@gmail.com', 
#     'rubaet.ema@gmail.com', 
#     'oyebodeoyewale065@gmail.com', 
#     'koyama.honami.kg7@naist.ac.jp', 
#     'sasaki.rina.su6@g.ext.naist.jp', 
#     'marina.alki@tum.de', 
#     # 'nandakumar.ardra.na1@bs.naist.jp'
# ]

# test_id =  'nandakumar.ardra.na1@bs.naist.jp'
# testfile_path = f'/Users/shigeyuki-t/Desktop/Experiment_2/analysis/subject/{test_id}/feature_{test_id}.csv'

# roc_df = pd.read_csv(testfile_path)
# roc_df = roc_df.drop(columns=['userEmail', 'dateTime', 'Unnamed: 0'])

# print(roc_df.columns)

# # タスクごとに予測を実行
# tasks = roc_df["task"].unique()  # ユニークな task 値を取得

# for task in tasks:
#     # 現在の task に対応するデータを取得
#     task_data = roc_df[roc_df["task"] == task].copy()
#     # task_data = roc_df[roc_df["task"].isin(range(task-10, task+1))].copy()ここじゃない

#     task_predictions = []  # このタスクの全モデルの予測を保存

#     for sensor_id in clustered_groups:
#         model_path = f'/Users/shigeyuki-t/Desktop/Experiment_2/analysis/subject/{sensor_id}'
# #         print(model_path)
#         for n in range(1, 151):
#             try:
#                 model_file = os.path.join(model_path, f'model_{n}.txt')
#                 if not os.path.exists(model_file):
#                     continue

#                 load_model = lgb.Booster(model_file=model_file)

#                 # モデルが期待する特徴量の確認
#                 expected_columns = load_model.feature_name()

#                 # モデルに必要な特徴量を補完
#                 missing_columns = list(set(expected_columns) - set(task_data.columns))
#                 for column in missing_columns:
#                     task_data[column] = 0
#                 task_data = task_data[expected_columns]  # 特徴量の順番をモデルに合わせる

#                 # 予測を実行
#                 pred = load_model.predict(task_data, num_iteration=load_model.best_iteration)
#                 task_predictions.append(pred)

#             except Exception as e:
#                 print(f"Failed to load model {n}: {e}")
#                 continue

#     # タスク単位の平均予測値を計算
#     if task_predictions:
#         task_predictions_array = np.array(task_predictions)  # shape: (モデル数, データ数)
#         mean_pred_task = np.mean(task_predictions_array, axis=0)

#         # タスクデータに予測値を反映
#         roc_df.loc[roc_df["task"] == task, "predict_isCorrect"] = mean_pred_task
#     else:
#         print(f"No predictions available for task {task}")

In [101]:
# # ROC曲線を使用して最適な閾値を計算
# if "isCorrect" in roc_df.columns and "predict_isCorrect" in roc_df.columns:
#     actual = roc_df["isCorrect"].values
#     predictions = roc_df["predict_isCorrect"].values
#     fpr, tpr, thresholds = roc_curve(actual, predictions)
#     optimal_idx = np.argmax(tpr - fpr)  # TPR - FPR が最大となる閾値
# #     # FPRが0.2以下の範囲でTPRが最大となる閾値を選択
# #     acceptable_fpr = 0.6
# #     filtered_indices = np.where(fpr <= acceptable_fpr)[0]
# #     optimal_idx = filtered_indices[np.argmax(tpr[filtered_indices])]
#     optimal_threshold = thresholds[optimal_idx]

#     print(f"Optimal threshold from ROC curve: {optimal_threshold:.4f}")

#     # 閾値を使用して最終予測を計算
#     roc_df["final_predict_isCorrect"] = roc_df["predict_isCorrect"].apply(lambda x: 1 if x > optimal_threshold else 0)

#     # 精度を計算
#     correct_predictions = (roc_df["final_predict_isCorrect"] == roc_df["isCorrect"]).sum()
#     accuracy = correct_predictions / len(roc_df)
#     print(f"predict :{roc_df['final_predict_isCorrect']}")
#     print(f"Overall Accuracy: {accuracy:.4f}")

#     # AUCの計算
#     roc_auc = auc(fpr, tpr)
#     print(f"AUC: {roc_auc:.4f}")
          
#     # F1スコアの計算
#     f1 = f1_score(actual, roc_df["final_predict_isCorrect"])
#     print(f"F1 Score: {f1:.4f}")
          
#     # 混同行列の計算
#     conf_matrix = confusion_matrix(actual, roc_df["final_predict_isCorrect"])
#     print(f"Confusion Matrix:\n{conf_matrix}")
          
#     # 混同行列の可視化
#     plt.figure(figsize=(6, 5))
#     sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=["Incorrect", "Correct"], yticklabels=["Incorrect", "Correct"])
#     plt.xlabel('Predicted Label')
#     plt.ylabel('True Label')
#     plt.title('Confusion Matrix')
#     plt.show()

#     # ROC曲線の描画
#     plt.figure(figsize=(8, 6))
#     plt.plot(fpr, tpr, label=f"ROC curve (AUC = {roc_auc:.2f})")
#     plt.scatter(fpr[optimal_idx], tpr[optimal_idx], color='red', label=f"Optimal Threshold = {optimal_threshold:.2f}")
#     plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
#     plt.title('ROC Curve')
#     plt.legend()
#     plt.grid()
#     plt.show()

#     # 全体の予測分布を可視化
#     plt.figure(figsize=(8, 6))
#     plt.hist(predictions, bins=30, alpha=0.7, label='Predictions')
#     plt.axvline(optimal_threshold, color='red', linestyle='--', label='Optimal Threshold')
#     plt.xlabel('Prediction Probability')
#     plt.ylabel('Frequency')
#     plt.title('Prediction Probability Distribution')
#     plt.legend()
#     plt.grid()
#     plt.show()

#     print("Predictions with dynamic threshold are saved in `final_predict_isCorrect`.")
# else:
#     print("Either `isCorrect` or `predict_isCorrect` column is missing.")

# 多数決

In [ ]:
# # 各モデルの予測結果を格納するリスト
# cross_predictions = []

# for sensor_id in group:
#     model_path = '/Users/shigeyuki-t/Desktop/Experiment_2/analysis/subject/' + f'{sensor_id}'
#     print(model_path)
#     for n in range(1,151):
#         try:         
#             model_file = os.path.join(model_path, f'model_{n}.txt')
#             if not os.path.exists(model_file):
#                 continue
            
#             load_model = lgb.Booster(model_file=model_file)
#             zikkenn = test_df[features].fillna(0).replace([np.inf, -np.inf], 0)
#             pred = load_model.predict(zikkenn, num_iteration=model.best_iteration)
#             cross_predictions.append(pred)

#         except Exception as e:
#             print(f"Failed to load model {n}: {e}")
#             continue

# # 多数決で予測
# if cross_predictions:
#     predictions_array = np.array(cross_predictions)  # shape: (モデル数, データ数)
#     # 各データポイントごとに予測結果を多数決で集約
#     majority_vote = np.round(np.mean(predictions_array, axis=0))
#     majority_vote = [1 if p > 0.5 else 0 for p in majority_vote]
#     print("多数決結果:", majority_vote)
# else:
#     print("No predictions available.")


In [ ]:
# Model results
accuracies = [
    0.6043, 0.5302, 0.4614, 0.4573, 0.4530, 
    0.5676, 0.3960, 0.5630, 0.5636, 0.5224
]
f1_scores = [
    0.6837, 0.6156, 0.3173, 0.5157, 0.5072, 
    0.6954, 0.5010, 0.6449, 0.6367, 0.6006
]

# Calculate means
mean_accuracy = sum(accuracies) / len(accuracies)
mean_f1_score = sum(f1_scores) / len(f1_scores)

mean_accuracy, mean_f1_score
